In [1]:
'''Importing Modules'''

'''
Prerequisites:
NumPy             https://numpy.org/doc/stable/
Matplotlib        https://matplotlib.org/stable/index.html
PyTorch           https://pytorch.org/docs/stable/index.html
Torchvision       https://pytorch.org/docs/stable/index.html
PIL               https://pillow.readthedocs.io/en/stable/
GitPython         https://gitpython.readthedocs.io/en/stable/
split-folders:    https://pypi.org/project/split-folders/
python-dotenv:    https://pypi.org/project/python-dotenv/
'''

# vanilla:
import os
import json
import time
from time import strptime
import datetime
from datetime import timedelta
import shutil
from collections import OrderedDict
import random
import sys
# external:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.autograd import Variable
import torchvision
from torchvision import datasets, transforms, models
import PIL.Image as Image
import git
import splitfolders
from dotenv import load_dotenv
# custom:
from routines import *
from displays import *
import myTransforms

println([('done.', 'g')])

done.               


In [49]:
'''Parsing the Input Directory Supplied by the User'''

valid_filetypes = ('.jpg', '.png', '.jpeg')
input_path = './input/'
files = []
bad_files = []

println([(f'looking for image files in \'{input_path}\'...', 'y')])


no_dir = False
no_files = False

try:
    for file in os.listdir(input_path):
        if file.endswith(valid_filetypes):
            files.append(file)
        else:
            bad_files.append(file)
except FileNotFoundError as e:
    no_dir = True

if no_dir:
    e_msg = 'cannot find the input directory (it must be named \'input\' and be in root)'
    raise SystemExit(e_msg)

if bad_files != []:
    e_msg = f'invalid files were found in input the directory, please remove them:\n{bad_files}'
    raise SystemExit(e_msg)

if files == []:
    e_msg = 'cannot find any image files in the input directory'
    raise SystemExit(e_msg)


println(['#', 'filename', 'height', 'width', 'color'], header=True)    
for idx, file in enumerate(files):
    img = Image.open(input_path + file)
    tensor = transforms.Compose([transforms.PILToTensor()])(img)
    color = 'rgb' if len(tensor)==3 else 'greyscale'
    height = len(tensor[0])
    width = len(tensor[0][0])
    println([idx + 1, file, height, width, color])


println([('done.', 'g')])

looking for image files in './input/'...
#                   filename            height              width               color               
1                   1.jpg               900                 1200                rgb                 
2                   2.png               427                 640                 greyscale           
done.               


In [ ]:
'''Instaciating Our Model'''

checkpoints_path = './checkpoints'

println([(f'looking for a \'.pth\' file in \'{checkpoints_path}\'...', 'y')])

try:
    for file in os.listdir(input_path):
        if file.endswith(valid_filetypes):
            files.append(file)
        else:
            bad_files.append(file)
except FileNotFoundError as e:
    no_dir = True